In [10]:
#connect to s3 with the sdk
from boto.s3.connection import S3Connection
from PIL import Image 
import webbrowser
import datetime
import boto3
from boto.s3.key import Key

from matplotlib.pyplot import imshow
import numpy as np
from PIL import Image
client = boto3.client('rekognition', 'us-east-1')
%matplotlib inline

#You must change these to your own password and bucket information.
conn = S3Connection()

from matplotlib.pyplot import imshow
import json
from pprint import pprint

%matplotlib inline
import matplotlib
matplotlib.rc("savefig", dpi=100)
import cv2
from collections import namedtuple

#only run this once or you will freeze and have to restart the notebook server
#to get this working properly, you'll need to change the 'conn = ' line in mxnet_mtcnn_face_detection
#from mxnet_mtcnn_face_detection.main_rekognition_filter import getdetector
#from mxnet_mtcnn_face_detection.main_rekognition_filter import countfaces
#change this to the location of your model folder
#detector=getdetector('mxnet_mtcnn_face_detection/model')

print ('done')

done


In [11]:
#print the bucket name and contents
mybuckets = conn.get_all_buckets()
print(len(mybuckets))

print ('print info about this bucket')
#this bucket is read-public, so feel free to follow this example code without changing 
#anything besides public/private key
bucketname='rekognitionbotoartifact'
s3bucket = conn.get_bucket(bucketname)
print(s3bucket)
size = 0
count = 0
for key in s3bucket.list():
    size += key.size
    count += 1
 
print ("%.3f GB" % (size*1.0/1024/1024/1024))
print ("num files: "+str(count))

for key in s3bucket.list():
        print ("{name}\t{size}\t{modified}".format(
                name = key.name,
                size = key.size,
                modified = key.last_modified,
                ))

2
print info about this bucket
<Bucket: rekognitionbotoartifact>
0.002 GB
num files: 23
Cano1.png	12302	2017-05-30T17:38:41.000Z
beachpic.png	92369	2017-06-21T14:30:49.000Z
canoandcruz.png	102642	2017-05-30T17:38:41.000Z
canoandjayz.png	50004	2017-05-30T17:38:41.000Z
canoandreyes.png	81886	2017-05-30T17:38:42.000Z
cat_picture.png	113078	2017-06-19T20:30:25.000Z
cruz.png	18921	2017-05-30T17:38:42.000Z
danes.jpg	17248	2017-07-31T15:29:50.000Z
dog_pic.png	56996	2017-06-19T20:30:27.000Z
embiidandokafor.png	94450	2017-05-30T17:38:42.000Z
embiidandsimmons.png	161768	2017-05-30T17:38:43.000Z
foursixers.png	77603	2017-05-30T17:38:44.000Z
hinkie.png	163536	2017-05-30T17:38:44.000Z
hinkieandbrown.png	48090	2017-05-30T17:38:45.000Z
hinkieandnoel.png	316380	2017-05-30T17:38:45.000Z
hinkieandsaric.png	228041	2017-05-30T17:38:46.000Z
hinkiembiid.png	101365	2017-05-30T17:38:47.000Z
hinkieofakor.png	245820	2017-05-30T17:38:47.000Z
kbacon.jpg	24958	2017-07-31T15:29:50.000Z
running_on_beach.png	60107	20

In [ ]:
#a demo of looping over a s3 bucket. printing out the MTCNN and the num_faces 
#from rekognition's api. Looking at printed output gives interesting results about errors
#coming from both sides. Errors are few though, and are usually worth taking in exchange
#for the cost savings
#import commands
count=0
failcount=0

correctAPICall=0
incorrectAPICall=0
correctHold=0
incorrectHold=0
        
for key in s3bucket.list():
    #use this line to control how many loops
    if count==50:
        break
    count += 1
    if ((count % 100) == 0):
        print ('at count: '+str(count)+'\n')
        
    print(key.name)
    try:
        
        response = client.recognize_celebrities(
            Image={
                'S3Object': {
                #marinus will need to change the bucket name to whichever they want to use
                'Bucket': bucketname,
                'Name': key.name
                #,'Version': '1'
            }
        }
        )
        
        #version 1: get ground truth from rekognition api
        num_api_faces=len(response['CelebrityFaces'])
        print('Rekognition faces detected count: '+str(num_api_faces))
        
    

        
        #keep track of success/fail ratio
        
    except:
        print ('failed on: '+key.name)
        print('')
        failcount=(failcount+1)
        
        



Cano1.png
Rekognition faces detected count: 1
beachpic.png
Rekognition faces detected count: 0
canoandcruz.png
Rekognition faces detected count: 2
canoandjayz.png
Rekognition faces detected count: 2
canoandreyes.png
Rekognition faces detected count: 1
cat_picture.png
Rekognition faces detected count: 0
cruz.png
Rekognition faces detected count: 1
danes.jpg
failed on: danes.jpg

dog_pic.png
Rekognition faces detected count: 0
embiidandokafor.png
Rekognition faces detected count: 2
embiidandsimmons.png
Rekognition faces detected count: 2
foursixers.png
Rekognition faces detected count: 4
hinkie.png
Rekognition faces detected count: 1
hinkieandbrown.png
Rekognition faces detected count: 2
hinkieandnoel.png
Rekognition faces detected count: 2
hinkieandsaric.png
Rekognition faces detected count: 2
hinkiembiid.png
Rekognition faces detected count: 2
hinkieofakor.png
Rekognition faces detected count: 2
kbacon.jpg
failed on: kbacon.jpg

running_on_beach.png
Rekognition faces detected count: 0


In [ ]:
#retrieve and print a selected image
image='embiidandsimmons.png'.replace('+',' ')
print (image)
#make sure that you have the folder path in whatever instance is running this code
folder='/home/ubuntu/MTCNN-face-filter'
temp_image=folder+'temp_image.png'

key = s3bucket.get_key(image)
#use the same file name for each temp image or you will start getting a ton of photos 
#stored in the folder
#also, if you are using your own bucket, you may need to tweak this code to handle '/'s
key.get_contents_to_filename(temp_image)                                                                  
pil_im = Image.open(temp_image, 'r')
imshow(np.asarray(pil_im))